# Stock market clustering

_Data Structures and Algorithms_

_Imperial College Business School_


---
This assignment is divided into three parts. In the first part, you will work on `pandas` data analysis. In the second part, you will implement a clustering algorithm to group companies based on their stock price movements. In the final part, you will explore ways to extend and improve this analysis. 

---
- **Assessment criteria**
  - Graded on **both correctness and presentation**.
  - Make results easy to read: apply **string formatting**, include **plots** when useful, and **comment your code**.

- **Testing**
  - There are **no OK tests** for this assignment.
  - You are expected to **explore the data and problem**, and use a **search engine** to identify appropriate pandas functions.
  - See `veryUseful.py` for a short list of potentially helpful pandas functions.

- **Collaboration**
  - This is **group work**; consider dividing tasks.
  - Some team members can focus on the **pandas/data analysis** component, others on the **algorithmic** component.
  - **Intermediate results** are provided to **test your algorithm**; you can begin both parts immediately (see **Question 3** for details).

- **Use of generative AI**
  - **Permitted only in Part 3** of the assignment.
  - If used (or if other external sources are used) in Part 3, **clearly document how** you used them.
  - **Not allowed** in the other parts.
---


## Your group

You'll complete this assignment in your assigned study groups. If you are unsure about your group, please contact the programme team.

## Submission
Submit your work via GitHub, following the detailed instructions provided in the assessment submission guidelines.


## Part 1: Pandas

**30% of grade**

In the previous homework, we used lists to study stock prices. The `pandas` library provides some more effective tools for data analysis.

The assignment comes with two files containing company data:
- `SP_500_firms.csv` with firm and ticker names
- `SP_500_close_2015.csv` with stock price data for 2015

Let's first load up this data.

In [ ]:
# Load data into Python

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

def read_names_into_dict():
    """
    Read company names into a dictionary
    """
    d = dict()
    with open("SP_500_firms.csv") as csvfile:
        input_file = csv.DictReader(csvfile)
        for row in input_file:
            #print(row)
            d[row['Symbol']] = [row['Name'],row['Sector']]
    return d

names_dict = read_names_into_dict()
comp_names = names_dict.keys()

# Read price data with pandas
filename = 'SP_500_close_2015.csv'
price_data = pd.read_csv(filename, index_col=0)

### Question 1: Returns

In the previous homework, we calculated stock price _returns_ over a period of time. The return is defined as the percentage change, so the return between periods $t-1$ and $t$ for stock price $p$ would be

$$
x_t = \frac{p_t - p_{t-1}}{p_{t-1}}.
$$

Calculate the returns in `pandas` for all the stocks in `price_data`.

In [ ]:
# Calculate company returns in this cell

### Question 1.1: Highest and lowest daily returns

Use pandas to find the 10 highest daily returns amongst all companies. Search online for what were the reasons behind the highest returns. Present your results in a clean and immediately readable form.

Repeat with the lowest daily returns.

In [ ]:
# Your code here

### Question 1.2: Highest and lowest yearly returns

Find the 10 highest yearly returns amongst all companies. Present your results in a clean and immediately readable form.

Repeat with the lowest yearly returns.

In [ ]:
# Your code here

### Question 1.3: Highest and lowest volatilities

Find the 10 highest yearly volatilities (standard deviations) amongst all companies. Present your results in a clean and immediately readable form.

Repeat with the lowest volatilities.

In [ ]:
# Your code here

### Question 2: Correlations

Analysts often care about the _correlation_ of stock prices between firms. Correlation measures the statistical similarity between the two prices' movements. If the prices move very similarly, the correlation of their _returns_  is close to 1. If they tend to make exactly the opposite movements (ie one price moves up and the other one down), the correlation is close to -1. If there is no clear statistical relationship between the movements of two stock prices, the correlation in their returns is close to zero.

For a sample of stock price returns $x,y$ with observations for $n$ days, the correlation $r_{xy}$ between $x$ and $y$ can be calculated as:

$$
r_{xy} = \frac{\sum x_i y_i - n \bar{x}\bar{y}}{ns_x s_y} = {\frac {n\sum x_{i}y_{i}-\sum x_{i}\sum y_{i}}{{\sqrt {n\sum x_{i}^{2}-(\sum x_{i})^{2}}}~{\sqrt {n\sum y_{i}^{2}-(\sum y_{i})^{2}}}}}.
$$

Here $\bar{x}$ refers to the average value of $x$ over the $n$ observations, and $s_x$ to its standard deviation.

Based on time series of the stock returns we just computed, we can calculate a  correlation value for each pair of stocks, for example between MSFT (Microsoft) and AAPL (Apple). This gives us a measure of the similarity between the two stocks in this time period.


Calculate all correlations between companies. You can search online for a `pandas` or `numpy` function that does this directly.

In [ ]:
# Your code here

### Question 2.1

Next, analyse the correlations between the companies:
- Define functions to print out the $n$ top and bottom correlated companies for any given company. 
- Use your functions to study the following companies in the tech sector: Amazon, Microsoft, Facebook, Apple, and Google. Comment on the results. Which (possibly other) companies are they most closely related to in terms of highest correlations? Would you have expected the results you see?

In [ ]:
# Your code here

## Part 2:  Clustering

**30% of grade**

In this part of the assignment, you will develop a clustering algorithm to study the similarity of different stocks. 

The general purpose of clustering analysis is dividing a set of objects into groups that are somehow "similar" to each other. It is a widespread tool used for exploratory data analysis in diverse fields in both science and business. For example, in marketing analytics, cluster analysis is employed to group consumers into segments based on their characteristics or _features_, such as age, post code, purchase history, etc. These features are somehow aggregated to compare the similarity between consumers. Based on this similarity, a clustering algorithm then divides the consumers into segments.

We will apply this idea on stock market data to identify groups of stocks that perform similarly over time. There are many reasons for grouping stocks together, such as analysing trading strategies, risk management, or simply presenting stock market information. Publicly traded companies are often grouped together by simple features such as the industry they operate in (eg tech companies or pharma companies), but here we'll take a data-driven approach, grouping together stocks that perform similarly over time. 

Cluster analysis is an umbrella term for many different algorithmic approaches. Here you'll develop one that's based on the concept of `greedy` algorithm design, specified below. You'll also have the opportunity to explore other approaches using Python libraries.

What is a good measure for stocks "performing similarly" to use for clustering. Let's use the one we just calculated: correlations in their returns. How can we use this similarity information for clustering? We now have access to all correlations between stock returns in S&P 500. We can think of this as a _graph_ as follows. The _nodes_ of the graph are the stocks (eg MSFT and AAPL). The _edges_ between them are the correlations, which we have just calculated between each stock, where the value of the correlation is the edge weight. Notice that since we have the correlations between all companies, this is a _dense_ graph, where all possible edges exist.

We thus have a graph representing pairwise "similarity" scores in correlations, and we want to divide the graph into clusters. There are many possible ways to do this, but here we'll use a _greedy_ algorithm design. The algorithm is as follows:

1. Sort the edges in the graph by their weight (ie the correlation), pick a number $k$ for the number of iterations of the algorithm
2. Create single-node sets from each node in the graph
3. Repeat $k$ times:
	1. Pick the graph edge with the highest correlation
	2. Combine the two sets containing the source and the destination of the edge
	3. Repeat with the next-highest weight edge
4. Return the remaining sets after the $k$ iterations 

What does the algorithm do? It first initializes a graph with no connections, where each node is in a separate set. Then in the main loop, it runs through the $k$ highest-weighted edges, and adds connections at those edges. This leads to sets being combined (or "merged"). The result is "groups" of stocks determined by the highest correlations between the stock returns. These are your stock clusters.

For example, suppose that the toy graph below represents four stocks: A,B,C,D and their return correlations. Suppose we pick $k=2$ and run the algorithm. 

<img src="cluster0.png" alt="cluster0" style="width: 200px;"/>


The algorithm would begin by initializing four separate sets of one node each: {A}, {B}, {C}, {D}. It would then first connect C and D because of their correlation 0.95, resulting in just three sets: {A}, {B}, and {C,D}. Then it would connect A and B, resulting in two sets of two nodes each: {A,B}, and {C,D}. These would be our clusters for $k=2$.

### Question 3: Implementing the algorithm

Your task is to implement the clustering algorithm using the functions below. First, for convenience in implementing the algorithm, let's create a list of the correlations from the pandas data. 

In [ ]:
def create_correlation_list(correl):
    """
    Creates a list of correlations from a pandas dataframe of correlations
    
    Parameters:
        correl: pandas dataframe of correlations
    
    Returns:
        list of correlations containing tuples of form (correlation, ticker1, ticker2)
    """
    n_comp = len(correl.columns)
    comp_names = list(correl.columns)
    # Faster if we use a numpy matrix
    correl_mat = correl.to_numpy()
    L = [] # create list
    for i in range(n_comp):
        for j in range(i+1,n_comp):
            L.append((correl_mat[i,j],comp_names[i],comp_names[j]))
    return L

edges = create_correlation_list(correl)

Next, let's turn to the algorithm itself. Consider the example above, repeated here.

<img src="cluster0.png" alt="cluster0" style="width: 200px;"/>


Suppose we pick $k=3$ and have sorted the edge list in step 1 of the algorithm. How should we represent the clusters in step 2? One great way is to use a dictionary where each _key_ is a node, and each _value_ is another node that this node "points to". A cluster is then a chain of these links, which we represent as a dictionary.

In step 2 of the algorithm, we start with four nodes that point to themselves, ie the dictionary `{'A':'A','B':'B','C':'C','D':'D'}`. When a node points to itself, it ends the chain. Here the clusters are thus just the nodes themselves, as in the figure below.

<img src="cluster1.png" alt="cluster1" style="width: 200px;"/>


Let's walk through the algorithm's next steps. We first look at the highest-weight edge, which is between C and D. These clusters will be combined. In terms of the dictionary, this means that one of them will not point to itself, but to the other one (here it does not matter which one). So we make the dictionary at `C` point to `D`. The dictionary becomes `{'A':'A','B':'B','C':'D','D':'D'}`.

<img src="cluster2.png" alt="cluster2" style="width: 200px;"/>


The next highest correlation is between A and B, so these clusters would be combined. The dictionary becomes `{'A':'B','B':'B','C':'D','D':'D'}`.

<img src="cluster3.png" alt="cluster3" style="width: 200px;"/>


The third highest correlation is between C and B. Let's think about combining these clusters using the dictionary we have. Looking up `B`, we get `B`: the node B is in the _bottom_ of the chain representing its cluster. But when we look up `C`, it points to `D`. Should we make `C` point to `B`? No - that would leave nothing  pointing at `D`, and `C` and `D` should remain connected! We could perhaps have `C` somehow point at both nodes, but that could become complicated, so we'll do the following instead. We'll follow the chain to the bottom. In the dictionary, we look up `C` and see that it points to `D`. We then look up `D` which points to itself, so `D` is the _bottom_ node. We then pick one of the bottom nodes `B` and `D`, and make it point to the other. We then have the dictionary `{'A':'B','B':'B','C':'D','D':'B'}`, and the corresponding clustering in the figure below.

<img src="cluster4.png" alt="cluster4" style="width: 200px;"/>


In other words, we'll keep track of clusters in a dictionary such that **each cluster has exactly one bottom node**. To do this, we need a mechanism for following a cluster to the bottom. You'll implement this in the function `find_bottom` below. The function takes as input a node and a dictionary, and runs through the "chain" in the dictionary until it finds a bottom node pointing to itself.

The other thing we'll need to do is combine clusters by connecting two nodes. This means taking the two nodes, finding the bottom node for each node's cluster, and making one point to the other. You'll implement this in the function `merge_clusters` below.

Finally, you'll need to set up the algorithm by sorting the correlations, and then looping through this merging $k$ times. You'll implement this in the function `cluster_correlations` below. This completes the algorithm.

But there is one more thing. If you only keep track of a dictionary like `{'A':'B','B':'B','C':'D','D':'B'}`, how do you actually find the clusters from the dictionary? A convenient way is to store some extra information: the "starting nodes" of each cluster to which no other node links. For example, above these "starting nodes" would include all nodes `A,B,C,D` in the beginning, but only `A` and `C` in the end. If we keep track of these, we can then write a function that starts from each such remaining "starting node", works through to the bottom, and creates the cluster along the way. You'll implement this in the function `construct_sets` below.

### Intermediary results

You can load a pre-computed set of results up to this point using the following commands.

In [ ]:
# Load intermediary results from a "pickle" file
# You can use these with your algorithm below
import pickle
file_name = 'cluster_correlations'
with open(file_name, "rb") as f:
    correl = pickle.load(f)
    edges = pickle.load(f)

firms = list(correl.columns)
print(firms[:10])
edges[:10]

### Clustering implementation

Complete the following functions to implement the clustering algorithm.

In [ ]:
def find_bottom(node, next_nodes):
    """
    Find the "bottom" of a cluster starting from node in dictionary next_nodes

    Parameters:
        node: starting node
        next_nodes: dictionary of node connections

    Returns:
        the bottom node in the cluster
    """
    # Your code here
    pass


def merge_sets(node1, node2, next_nodes, set_starters):
    """
    Merges the clusters containing node1, node2 using the connections dictionary next_nodes.
    Also removes any bottom node which is no longer a "starting node" from set_starters.

    Parameters:
        node1: first node the set of which will be merged
        node2: second node the set of which will be merged
        next_nodes: dictionary of node connections
        set_starters: set of nodes that "start" a cluster

    Returns:
        does this function need to return something?
    """
    # Your code here


def cluster_correlations(edge_list, firms, k=200):
    """
    A mystery clustering algorithm
     
    Parameters:
         edge_list - list of edges of the form (weight,source,destination)
         firms - list of firms (tickers)
         k - number of iterations of algorithm

    Returns:
         next_nodes - dictionary to store clusters as "pointers"
            - the dictionary keys are the nodes and the values are the node in the same cluster that the key node points to
         set_starters - set of nodes that no other node points to (this will be used to construct the sets below)

    Algorithm:
         1 sort edges by weight (highest correlation first)
         2 initialize next_nodes so that each node points to itself (single-node clusters)
         3 take highest correlation edge
            check if the source and destination are in the same cluster using find_bottom
            if not, merge the source and destination nodes' clusters using merge_sets
         4 if max iterations not reached, repeat 3 with next highest correlation
         (meanwhile also keep track of the "set starters" ie nodes that have nothing pointing to them for later use)
    """
    # Sort edges
    sorted_edges = _____
    # Initialize dictionary of pointers
    next_nodes = {node: node for node in firms}
    # Keep track of "starting nodes", ie nodes that no other node points to in next_nodes
    set_starters = {node for node in firms}

    # Loop k times
    for i in range(k):
        # Your algorithm here
        
    return set_starters, next_nodes

Once we've run the algorithm, we'll need to construct the clusters. You can use the function below to do so.

In [ ]:
def construct_sets(set_starters, next_nodes):
    """
    Constructs sets (clusters) from the next_nodes dictionary
    
    Parameters:
        set_starters: set of starting nodes 
        next_nodes: dictionary of connections
    
    Returns: 
        dictionary of sets (clusters):
            key - bottom node of set; value - set of all nodes in the cluster
    
    """
    # Initialise an empty dictionary 
    all_sets = dict()
    
    # Loop:
    # Start from each set starter node
    # Construct a "current set" with all nodes on the way to bottom node
    # If bottom node is already a key of all_sets, combine the "current set" with the one in all_sets,
    # Otherwise add "current set" to all_sets
    for s in set_starters:
        cur_set = set()
        cur_set.add(s)
        p = s
        while next_nodes[p] != p:
            p = next_nodes[p]
            cur_set.add(p)
            
        if p not in all_sets:
            all_sets[p] = cur_set
        else: 
            for item in cur_set:
                all_sets[p].add(item)
    return all_sets

all_clusters = construct_sets(set_starters,next_nodes)

### Question 3.2: analysing the results

After you have implemented the algorithm in Python, add cells below answering the following questions:
- Do some detective work: what is the algorithm that you've implemented called? In what other graph problem is it often used? How are the problems related? (Hint: the algorithm is mentioned on the Wikipedia page for greedy algorithms.)
- Run the algorithm and present the results formatted in a useful way. 
- Discuss the results for different values of $k$.  
- Do the resulting clusters "make sense"? (You may need to search online what the companies do.) Verify that the stocks in your clusters perform similarly by plotting the returns and the (normalised) stock prices for some of the clusters.
- You may use graphs etc. to present your results.

## Part 3: 

**40% of grade**

Depending on your interests, you may work on either subsection below, or both. You might go deeper into one question than another, but for an outstanding grade, you should have at least some discussion on both.

You may use generative AI such as chatGPT to help with this part of the assignment (but not the other parts).

### In-depth analysis

The project is _open_ in the sense that you can probably think of further interesting questions to look into based on returns, correlations, and clusters. This is not required but being creative and going further than the above questions will make your work stand out. You can explore one or several of the ideas below, or come up with questions of your own.

Depending on your interests, you might look at different things. For example, when researching the algorithm, you might be interested in its complexity, and how to improve your implementation's efficiency. On Wikipedia, you may find a couple of ways to drastically improve the algorithm speed, but are relatively small changes to your code.

If you're more interested in the financial applications of clustering, there are also opportunities to think about further steps. For example, some people claim that you can derive trading strategies based on clustering - that often one of the stocks in a cluster is a leader and the others follow that price. If this is true, you could track the price of the leader stock and then trade the other stocks in the cluster based on changes in the leader's price. Do you think this would make sense? Do you have an idea on how to identify a leader stock?

You might also want to repeat the analysis for different time periods. You would be able to do this by looking at the code for the second homework to figure out how to read data from Yahoo Finance using pandas, and going through the process for all companies in the csv file for another time period. Perhaps you could explore for example how correlations between companies have changed over time, or how clusters found by your algorithm change over time.

### Exploring other clustering methods

You've used just one approach to clustering, and arguably not the best one. Research clustering algorithms and libraries to apply them in Python. Discuss some other algorithms that could be used, and how they differ from the one you've implemented. Look at the Python library `scikit-learn`. How would you apply the clustering algorithms provided by the library to stock price data? Would you need to develop new metrics other than correlations? If you want to go even further,  try running some of these other clustering algorithms on your data, and report the results. Start from here: http://scikit-learn.org/stable/modules/clustering.html#clustering; you'll find a stock market example there too. For future reference, you may also find other interesting machine-learning tools for both stock market analysis or other analytics purposes.

### Question 4

Create cells below to add your extra part as code and narrative text explaining your idea and results.

## Exploring other Clustering Methods

### Single Link Hiearchical Clustering issue

From part 2 of the assignment, the clustering method used there was a single-link hierarchical clustering. Essentially, this method turns correlation to a Euclidean distance measure. 

However, this method would face issues such as the effect of chaining. As the stocks are added one by one on to each other, like a chain. Since it only worries it self with the ditance between two points that are next to one another and not how the new stock links with all the stocks in the cluster already, the clusters created might not be robust. The single link method is also subject to noise and outliers. With daily returns being the data used, there could be a lot of idiosyncratic noise, which would affect the correlation estimates. This would lead to not ideal clustering, or clustering that is very unstable. The single link would also lead to the dominance of one cluster, as it continues to add on stocks of similar correlations, leaving very few number of clusters and a large imbalance of cluster sizes. 

### Other potential clustering methods and their limitations

<b>K Means and Spectral Clustering</b>

K Means Clustering and Spectral Clustering would provide more balanced clusters compared to the single link hierarchical clustering method. However, both of these methods require selecting a pre-determined number of clusters, and if we do not select the right number, we might over or under fit the clusters. 

<b>DBSCAN</b>

Density based Spatial Clustering of Applications with Noise would group the stock based upon the density of data points. It doesn't require specifying the number of clusters upfront, but the epsilon radius must be chosen, and this value would be applied for all clusters, which might not be realistic as clusters would differ in densities. 

<b>OPTICS</b>

Ordering points to Identify Clustering Structure is another algorithm based on density. In comparison to DBSCAN, OPTICS doesn't require a single general epsilon cutoff value. However, it is difficult to implement and interpret, and might not provide a neat list of clusters when not paired with an extraction method. 

<b>HDBSCAN</b>

Hierarchical DBSCAN evolves the DBSCAN algorithm by exploring clusters at all density levels and selects the most stable clusters. The epsilon cutoff is a range of values. The main limitation of the HDBSCAN is that it is more computational, which would be an issue for a large dataset. 


### HDBSCAN

With less than 500 stocks in the S&P500 2015 Close Price dataset provided, the HDBSCAN limitation would not apply too heavily here, and would be feasible to implement. The HDBSCAN finds the number of clusters, just like DBSCAN and OPTICS, and is able to tackle varying density clusters by varying the epsilon. 

HDBSCAN is able to identify noise datapoints as well. It does this when the points do not fit in any clusters, and leaves the stocks as individual which could be useful information too. We will implement the HDBSCAN to the 2015 stock price data. 

In [1]:
!pip install hdbscan
import pandas as pd
import numpy as np
import hdbscan

# Load the S&P500 close prices data
price_data = pd.read_csv('SP_500_close_2015.csv', index_col=0)

# Calculate daily returns
returns = price_data.pct_change()
returns = returns.dropna(axis=0)
returns = returns.dropna(axis=1)

# Calculate the correlation and distance matrix from the daily returns
correlation_matrix = returns.corr()
distance_matrix = np.sqrt(2 * (1 - correlation_matrix))

# Running HDBSCAN
clusterer = hdbscan.HDBSCAN(metric='precomputed', min_cluster_size=10)
clusterer.fit(distance_matrix)

# Add cluster labels to list of stocks
labels = clusterer.labels_
clustered_stocks = pd.DataFrame({'Ticker': correlation_matrix.index, 'Cluster': labels})

# No. of noise points
num_clusters = len(np.unique(labels)) - 1
num_noise = np.sum(labels == -1)

print(f"No. of clusters found: {num_clusters}")
print(f"No. of noise points: {num_noise}")

# Display stocks in each cluster
for cluster_num in range(num_clusters):
    cluster_tickers = clustered_stocks[clustered_stocks['Cluster'] == cluster_num]['Ticker'].tolist()
    print(f"\nCluster {cluster_num}:")
    print(", ".join(cluster_tickers))

Defaulting to user installation because normal site-packages is not writeable
No. of clusters found: 4
No. of noise points: 357

Cluster 0:
APC, APA, CVX, XEC, CXO, COP, DVN, EOG, XOM, HAL, HP, HES, MRO, MUR, NFX, NBL, OXY, PXD, SLB

Cluster 1:
MMM, ABT, ACN, AMG, AFL, A, AIG, AMP, AME, AON, ADP, BAC, BBT, BRK-B, BLK, SCHW, CB, CHD, CINF, C, CFG, CTSH, CL, CMA, DHR, DFS, ETFC, ETN, FITB, FISV, BEN, GD, GPC, GS, HIG, HSIC, HON, HBAN, ITW, IVZ, JNJ, JPM, KEY, KMB, LM, LNC, L, MTB, MMC, MA, MET, MCO, MS, NTRS, NOC, PH, PAYX, PBCT, PEP, PNC, PPG, PFG, PGR, PRU, RF, COL, SNA, SWK, STT, STI, TROW, BK, TRV, TMO, TMK, USB, UNM, WAT, WFC, XYL, ZION

Cluster 2:
AIV, AVB, BXP, EQR, ESS, EXR, FRT, GGP, HCP, KIM, PLD, PSA, O, SPG, SLG, UDR, VNO, HCN

Cluster 3:
LNT, AEE, AEP, AWK, CMS, ED, D, DTE, DUK, EIX, ETR, ES, PCG, PNW, PPL, PEG, SCG, SRE, SO, WEC, XEL


In [2]:
from sklearn.metrics import davies_bouldin_score

# Noise excluded (equal -1)
if num_clusters > 1:
    non_noise_indices = np.where(labels != -1)[0] 
    filtered_labels = labels[non_noise_indices]
    filtered_returns = returns.iloc[:, non_noise_indices]
    score = davies_bouldin_score(filtered_returns.T, filtered_labels)
    print(f"DBI score (excluding noise): {score:.4f}")
else:
    print("Erroor, less than 2 Clusters).")

DBI score (excluding noise): 1.2578


Other than the correlation matrix, HSDBSCAN also required a distance matrix, which can be easily calculated from the correlation matrix. A Davies-Bouldin Index(DBI) Score was calculated to observe how good the clusters are. The score is calculated based upon the ratio of the average distance of the points within the same cluster to the distance between the clusters themselve. A lower DBI score is better, with 0 being perfect clustering. DBI is a great metric, as it concerns itself with the seperation of the clusters, instead of the shape. 

THe DBI score for the HSDBSAN algorithm is 1.2578. This indicates that it is a decently effective clustering algorithm in this scenario as it is close to 1, meaning that the distance within the cluster is just slightly higher than the distance to the next cluster. This score could suggest some overlap between clusters exist as it is not close to 0. Even though the DBI score suggests that it was not great at clustering the S&P500 2015 Close Price data, the stock market is very complex, and could still provide some insights. 

## In depth Analysis: Leader Follower Momentum

In this section, we will identify lead-lag relationships, being when the movement of the price of the "leader" occurs before other securities. With the identification of such relationships, there is the potential to take advantage on the lagged reaction.

Financial literature has showed that identifying a pair of stocks using Granger Causality tests can provide a prediction of one stock's out of sample returns from the other stock's returns. This could work due to positive or negative announcements on a "leader" stock impacting the following stocks. 

To identify leader-follower pairs, we wil calculate lagged cross-correlation of returns and implement Granger Causality tests.  We set thresholds for correlation over 0.4 and p value of < 0.01 for the Granger test to identify strong relationships. The relationship can be tested out with the 2016 data. 

### Generative AI usage in this part
Generative AI was used to help implement the idea of making this a trading strategy once leaders and followers were identified, as their price movement theoretically would occur one after the other. GenAI helped with extracting the 2016 year data simply and clearly, and to also suggest which libraries to import to help the code go forward. It also helped with the ideas on how to show the success in the strategy. 



In [3]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import yfinance as yf

Firstly, we start off by loading in the provided S&P500 2015 Close Prices and downloading the 2016 equivalent through Yahoo Finance. 

In [5]:
# Loading 2015 Close Price 
price_2015 = pd.read_csv("SP_500_close_2015.csv")
price_2015.index = pd.to_datetime(price_2015.index)
tickers = list(price_2015.columns)
tickers_2015 = list(price_2015.columns[1:])
print("No. tickers:", len(tickers))

No. tickers: 497


In [6]:
def clean_tickers(tickers):
    cleaned_tickers = []
    for t in tickers:
        s = str(t).strip().upper().replace('.', '-')
        if s and s != 'DATE':
            cleaned_tickers.append(s)
    return cleaned_tickers

def download_prices(tickers, start_date, end_date):
    cleaned = clean_tickers(tickers)
    
    if not cleaned:
        print("No valid tickers provided.")
        return pd.DataFrame()

    data = yf.download(
        cleaned,
        start = start_date,
        end = end_date,
        auto_adjust = True, # Gets the adjusted close price
        progress = False # keeping output clean
    )
    
    if data.empty:
        print("Could not download any data for the given tickers.")
        return pd.DataFrame()
    prices = data['Close']
    prices.dropna(axis='columns', how='all', inplace=True)
    
    return prices

# Making 2016 dataset
price_2016 = download_prices(tickers_2015, start_date="2016-01-01", end_date="2016-12-31")
print(f"Found {price_2016.shape[1]} tickers in 2016, based upon 2015 data.")


95 Failed downloads:
['PBCT', 'ANTM', 'GPS', 'COG', 'TSS', 'NLSN', 'FBHS', 'BLL', 'JNPR', 'AGN', 'HCP', 'LM', 'CHK', 'TMK', 'PXD', 'CBS', 'HES', 'FISV', 'DISCK', 'ADS', 'NBL', 'TIF', 'JEC', 'CTXS', 'PDCO', 'MRO', 'FTR', 'SYMC', 'CXO', 'KSU', 'HRS', 'DISCA', 'ETFC', 'CELG', 'MNK', 'VIAB', 'DFS', 'LLL', 'UTX', 'CERN', 'VAR', 'FLIR', 'RHT', 'ENDP', 'XEC', 'PKI', 'JWN', 'RTN', 'YHOO', 'XLNX', 'APC', 'ATVI', 'SWN', 'MON', 'ABC', 'DO', 'XL', 'DLPH', 'CTL', 'SRCL', 'MYL', 'ALXN']: YFTzMissingError('possibly delisted; no timezone found')
['SNI', 'KORS', 'GGP', 'TYC', 'HCN', 'WFM', 'LLTC', 'SPLS', 'STJ', 'RAI', 'PCLN', 'LVLT', 'BCR', 'TSO', 'DPS', 'LUK', 'CBG', 'MJN', 'WYN']: YFPricesMissingError('possibly delisted; no price data found  (1d 2016-01-01 -> 2016-12-31)')
['IR', 'COH', 'EMC', 'FOX', 'DNB', 'FB', 'PX', 'CA', 'LB', 'SE', 'DOW', 'FOXA', 'BHI', 'STI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2016-01-01 -> 2016-12-31) (Yahoo error = "Data doesn\'t exist for st

Found 401 tickers in 2016, based upon 2015 data.


/tmp/ipykernel_822/2194958214.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices.dropna(axis='columns', how='all', inplace=True)


In [7]:
# Date as index:
if 'Date' in price_2015.columns:
    price_2015 = price_2015.set_index('Date')
if 'Date' in price_2016.columns:
    price_2016 = price_2016.set_index('Date')

Since we will be working across both datasets of 2015 and 2016, we will only work with companies that are present in both. Present in both meaning having the same ticker name for the two year span. This would allow us to identify the leader and follower pairs in 2015, and apply the idea of leader follower momentum into 2016. 

In [8]:
# Filtering 2015 dataset tickers to be the same as the 2016 ones we pulled
valid_tickers_2016 = list(price_2016.columns)
price_2015 = price_2015[valid_tickers_2016]

Next, we will calculate the daily returns for both datasets. Not only do we calculate the returns, but also the standard deviation of the daily returns for 2015. Finding the mean and sd of 2015 returns would allow us to compute z-score for 2016 relative to the 2015 distribution, allowing us to identify the abnormal price movements, which would signal potential trading options. 

The assumption needed with this method is that each stock's distribution of daily returns in 2015 is similar in 2016, which might not be very realistic as distributions can change.

In [9]:
# Daily returns
ret_2015 = price_2015.pct_change().dropna(how='all')
ret_2016 = price_2016.pct_change().dropna(how='all')

ret_2015 = ret_2015.loc[:, ret_2015.columns.notna()]
ret_2016 = ret_2016.loc[:, ret_2016.columns.notna()]

# Compute 2015 mean and sd
ret_mean_2015 = ret_2015.mean()
ret_std_2015  = ret_2015.std(ddof=0)  # population std deviation

# Compute z-scores for 2016 returns using 2015 stats
zscore_2016 = (ret_2016 - ret_mean_2015) / ret_std_2015

/tmp/ipykernel_822/360304046.py:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret_2016 = price_2016.pct_change().dropna(how='all')


We will now proceed with identifying the leader follower pairs in the 2015 dataset. Two different criterias will be needed to identify a strong leader follower pair. 

1) Lagged Cross-Correlation: 
    For each and every pair of stock, the code below will compute the correlation between the return of Stock 1 on day t, and Stock 2 and day t+1. The threshold for this criteria would be greater than or equal to 0.4, which is quite a strong correlation value for daily lagged relationships. Correlation also needs to be positive here, as we are looking for a momentum effect, where the stock prices move in the same direction.

2) Granger Causality Test:
    Once the pair has made it pass the correlation threshold, we test whether Stock 1 granger-causes Stock 2's returns in the next lag. Granger Causality is used here as it tests whether Stock 1 provides information that is statistically significant to Stock 2's returns at t+1. The null hypothesis in this case is that it does not granger cause. To reject that null hypothesis, the p value threshold is selected at less than or equal to 0.01. 

In [10]:
import itertools
from statsmodels.tsa.stattools import grangercausalitytests

tickers = list(ret_2015.columns)
tickers = [t for t in ret_2015.columns if t != 'Date']

# Identify pairs with lagged correlation ≥ 0.4
leaders = []
followers = []
lagged_corr_values = []

for A, B in itertools.permutations(tickers, 2):
    # Compute correlation of A_t(iloc -1) vs B_{t+1} (iloc 1)
    series_A = ret_2015[A].iloc[:-1]
    series_B_next = ret_2015[B].iloc[1:]
    corr = series_A.corr(series_B_next)

    if corr >= 0.40:
        leaders.append(A)
        followers.append(B)
        lagged_corr_values.append(corr)

print(f"{len(leaders)} pairs with correlation ≥ 0.4")

#Granger causality filter (lag=1)
leader_follower_pairs = []

for i, (A, B) in enumerate(zip(leaders, followers)): #A for dependent, B for independent
    series_A = ret_2015[A].iloc[:-1].values
    series_B_next = ret_2015[B].iloc[1:].values
     # Combine into array for statsmodels lib
    data = np.column_stack([series_B_next, series_A])
    # Skip if NaNs appear
    if np.isnan(data).any():
        continue
    # Performing test
    result = grangercausalitytests(data, maxlag=1, verbose=False)
    p_value = result[1][0]['ssr_ftest'][1]  # extract p-value for lag=1
    # Checking Statistical Sig
    if p_value <= 0.01:
        leader_follower_pairs.append((A, B, lagged_corr_values[i], p_value))
print(f"No. pairs after Granger test: {len(leader_follower_pairs)}")

# Sort by correl coef, highest to lowest
leader_follower_pairs.sort(key=lambda x: x[2], reverse=True)
# Display top 5 pairs
print("\nTop 5 Leader–Follower pairs:")
for A, B, corr, p in leader_follower_pairs[:5]:
    print(f"{A} → {B} | Corr={corr:.3f}, p={p:.4f}")


71944 pairs with correlation ≥ 0.4


/home/vscode/.local/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


No. pairs after Granger test: 2025

Top 5 Leader–Follower pairs:
MET → PRU | Corr=0.914, p=0.0075
KEY → BBT | Corr=0.904, p=0.0085
BBT → KEY | Corr=0.904, p=0.0013
USB → WFC | Corr=0.902, p=0.0045
ZION → CMA | Corr=0.902, p=0.0075


We have now identified the pairs of Leader and Follower stocks. With this information, we can create a return strategy based on these pairs, and apply it into the 2016 dataset. 

With the z-scores calculated previously, which identified whether the stock had experienced abnormal returns based on the 2015 data, we create a signal that if the leader stock was the one that experienced it, we take it as a signal for its follower(s).

After the signals have been identified, we can enter a trade with the follower stock, with the assumption that we can do it at the start of the day, before the close price has been reached. This trade follows the idea the the following stock would have the same direction price reaction of the signal identified. 

The number of trades are also capped at 5 per day. A limit set on the amount of trades would allow us to only trade the largest abnormal movements of the leader's closing price at time t. This would allow us to avoid trades that are just random movements of the stock price.

In [11]:
#2016 implementation
z_threshold = 2.5 #a threshold set for making a trade, when the leader's close price is 2.5 sd away from its mean. 2.5 is a good baseline, should occur only around 1% of the time
min_corr = 0.4
max_p = 0.01
max_trades_per_day = 5 #preventing taking too many trades a day
slippage_bp = 10 #putting a trading cost of 10 basis points to account for transaction costs and/or price slippage

# Filter leader follower pairs
leader_to_followers = {}
for A, B, corr, p in leader_follower_pairs:
    if corr >= min_corr and p <= max_p:
        leader_to_followers.setdefault(A, []).append(B)

strategy_ret = pd.Series(0.0, index=ret_2016.index)
trades = []

for date in ret_2016.index[:-1]:
    next_day_idx = ret_2016.index.get_indexer([date])[0] + 1
    if next_day_idx >= len(ret_2016.index):
        break
    next_day = ret_2016.index[next_day_idx]
    signals = []

    for leader, followers_list in leader_to_followers.items():
        if pd.isna(zscore_2016.loc[date, leader]):
            continue
        z = zscore_2016.loc[date, leader]
        if abs(z) >= z_threshold and abs(zscore_2016.shift(1).loc[date, leader]) >= z_threshold:
            pos = +1 if z >= z_threshold else -1
            for foll in followers_list:
                signals.append((leader, foll, pos))

    # Rank and cap signals per day
    signals.sort(key=lambda x: abs(x[2]), reverse=True)
    if len(signals) > max_trades_per_day:
        signals = signals[:max_trades_per_day]

    # Execute trades
    if not signals:
        continue
    N = len(signals)
    daily_return = 0.0
    for leader, foll, pos in signals:
        if pd.isna(ret_2016.loc[next_day, foll]):
            continue
        follower_ret = ret_2016.loc[next_day, foll]
        trade_ret = pos * follower_ret - slippage_bp/10000
        daily_return += trade_ret / N
        trades.append({
            "Date": date, "Leader": leader, "Follower": foll, "Leader_z": z,
            "Follower_ret": follower_ret, "Position": "Long" if pos==1 else "Short",
            "Trade_return%": trade_ret*100
        })
    strategy_ret[next_day] = daily_return


With the trades implemented, we examine our overall performance over the whole year of 2016. The cumulative return is calculated, to observe the overall impact of the trades made. 

The Sharpe Ratio is also calculated, which suggests how good the returns received were while accounting for the volatility. Ideally, a Sharpe Ratio over 1 would be decent. 

Other metrics are provided, such as the total amount of trades made over the year and the average return per trade. The Hit Rate would show the percentage of the toal trades that provided a positive return. 

In [ ]:
#Strategy Performance
# Convert strategy daily returns to a DataFrame
strategy_ret = strategy_ret.fillna(0.0)
cum_return = (1 + strategy_ret).cumprod() - 1

# Performance metrics
total_return = cum_return.iloc[-1]
annualised_ret = (1 + total_return) ** (252/len(strategy_ret)) - 1
annualised_vol = strategy_ret.std(ddof=0) * np.sqrt(252)
sharpe = (strategy_ret.mean() / strategy_ret.std(ddof=0)) * np.sqrt(252) if strategy_ret.std(ddof=0)!=0 else 0.0

# Trade numbers
trades_df = pd.DataFrame(trades)
hit_rate = (trades_df['Trade_return%'] > 0).mean() * 100  # % of trades with positive return
avg_trade_ret = trades_df['Trade_return%'].mean()
num_trades = len(trades_df)

print(f"Cumulative Return: {total_return*100:.2f}%")
print(f"Annualised Return: {annualised_ret*100:.2f}%,  Volatility: {annualised_vol*100:.2f}%,  Sharpe Ratio: {sharpe:.2f}")
print(f"Total Trades: {num_trades},  Hit Rate: {hit_rate:.1f}%,  Avg Trade Return: {avg_trade_ret:.2f}%")


Cumulative Return: 25.16%
Annualised Return: 25.27%,  Volatility: 12.43%,  Sharpe Ratio: 1.87
Total Trades: 271,  Hit Rate: 54.6%,  Avg Trade Return: 0.36%


<b>Comments on Trade Strategy Results</b>

Firstly, an annualised return of 25.27% is great, and suggests that the strategy is able to generate profit, even when accounting for the volatility of 12.43%

The Sharpe Ratio is a measure of risk-adjusted returns, and a ratio of 1.87 is also excellent. This means that it is able to generate returns with relatively low volatility. 

With more than a trade a day made over the 2016 year (trading days), the hit rate of 54.6% is great, showing us that more trades were won than lost overall, with the average trade return also being positive at 0.36%.

Running the strategy and changing its inputs could allow us to get better results. Trade caps can be altered to be even smaller to limit the amount of trades, paired along with making the z threshold even higher, for trades of even better quality with higher price movements from the leader as the signal


## All done!

Make sure to follow submission guidelines.